In [2]:
from mongodb import load_data
import pandas as pd
import numpy as np
import re
import datetime
from langdetect import detect_langs

In [3]:
def clean_text(text):
    if text == None:
        return None
    text = text.lower()
    text = re.sub('http[^ ]* ', '', text)
    text = re.sub('www[^ ]* ', '', text)
    text = re.sub('@[^ ]* ','', text)
    text = re.sub('[-!@#$%^&*()_+=/?,.<>\|{};:`~]', ' ', text)
    text = re.sub('[^a-zA-Z ]', '', text)
    text = text.strip().split()
    if len(text) < 3:
        return None
    return " ".join(text)

In [4]:
def get_date(created):
    time = datetime.datetime.fromtimestamp(created)
    return time.date()

In [5]:
def detect_language(text):
    try:
        langs = detect_langs(text)
        lang = langs[0].lang
        return lang
    except:
        return None

In [7]:
df = load_data()

df = df.loc[df['subreddit'] == 'worldnews']

df = df[df['created_utc'].notnull()]

df['date'] = df['created_utc'].apply(get_date)

df['log_score'] = np.log10(1 + df['score'])

In [8]:
df['lang'] = df['title'].apply(detect_language)

In [9]:
#df.to_csv("sample.csv", encoding='utf-8')

In [12]:
df = pd.read_csv("sample.csv", encoding='utf-8')

/home/qara/anaconda3/envs/woong/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,7,8,10,11,15,16,19,20,23,24,25,26,27,31,32,35,37,40,42,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df = df[df['lang'] == 'en']

In [14]:
df['title'] = df['title'].apply(clean_text)

In [15]:
df = df[df['title'].notnull()]
df = df.reset_index(drop=True)

df = df[['title', 'score', 'log_score', 'date']]

In [18]:
df.to_csv("reddit_worldnews_1year.csv", index = False)